In [1]:
import pandas as pd

# Load the main data file (adjust name to match your Excel file exactly)
df = pd.read_excel("data/Analytics and Forecasting - Calls Data.xlsx", sheet_name="Data")

df.head()


,DAY,CALLS HANDLED,CALLS MISSED,QUEUE,VENDOR_COMPANY,SATISFIED,CALLS WITHIN SLA,IS OUTAGE ?
0,2024-01-01,1060.5,73.5,Customer Queue,Vendor A,460,986.986029,True
1,2024-01-01,2151.0,73.5,Customer Queue,Vendor B,235,1789.695254,False
2,2024-01-01,6667.5,322.5,Customer Queue - Vendor B,Vendor B,4297,6214.145123,False
3,2024-01-01,10255.5,63.0,Customer Queue - Vendor A,Vendor A,4099,9004.283723,True
4,2024-01-02,718.5,13.5,Customer Queue,Vendor A,125,650.914783,True


In [2]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1597 entries, 0 to 1596
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   DAY                1597 non-null   datetime64[ns]
 1   CALLS HANDLED      1597 non-null   float64       
 2   CALLS MISSED       1597 non-null   float64       
 3   QUEUE              1597 non-null   object        
 4   VENDOR_COMPANY     1597 non-null   object        
 5   SATISFIED          1597 non-null   int64         
 6   CALLS WITHIN SLA   1597 non-null   float64       
 7   IS OUTAGE ?        1597 non-null   bool          
dtypes: bool(1), datetime64[ns](1), float64(3), int64(1), object(2)
memory usage: 89.0+ KB


In [3]:
# Make a copy to work with
data = df.copy()

# Ensure DAY is datetime
data['DAY'] = pd.to_datetime(data['DAY'])

# Create helper columns
data['DATE'] = data['DAY'].dt.date
data['MONTH'] = data['DAY'].dt.month
data['MONTH_NAME'] = data['DAY'].dt.strftime('%B')
data['DAY_OF_WEEK'] = data['DAY'].dt.day_name()
data['QUARTER'] = data['DAY'].dt.quarter
data['TOTAL_CALLS'] = data['CALLS HANDLED'] + data['CALLS MISSED']

data.head()


,DAY,CALLS HANDLED,CALLS MISSED,QUEUE,VENDOR_COMPANY,SATISFIED,CALLS WITHIN SLA,IS OUTAGE ?,DATE,MONTH,MONTH_NAME,DAY_OF_WEEK,QUARTER,TOTAL_CALLS
0,2024-01-01,1060.5,73.5,Customer Queue,Vendor A,460,986.986029,True,2024-01-01,1,January,Monday,1,1134.0
1,2024-01-01,2151.0,73.5,Customer Queue,Vendor B,235,1789.695254,False,2024-01-01,1,January,Monday,1,2224.5
2,2024-01-01,6667.5,322.5,Customer Queue - Vendor B,Vendor B,4297,6214.145123,False,2024-01-01,1,January,Monday,1,6990.0
3,2024-01-01,10255.5,63.0,Customer Queue - Vendor A,Vendor A,4099,9004.283723,True,2024-01-01,1,January,Monday,1,10318.5
4,2024-01-02,718.5,13.5,Customer Queue,Vendor A,125,650.914783,True,2024-01-02,1,January,Tuesday,1,732.0


In [6]:
# total_calls & total_handled
total_calls = data['TOTAL_CALLS'].sum()
total_handled = data['CALLS HANDLED'].sum()

# total missed (shared counted once)
shared = data[data['QUEUE'] == 'Customer Queue']
vend_a_q = data[data['QUEUE'] == 'Customer Queue - Vendor A']
vend_b_q = data[data['QUEUE'] == 'Customer Queue - Vendor B']

shared_missed_actual = shared['CALLS MISSED'].sum()
vend_a_missed_ded = vend_a_q['CALLS MISSED'].sum()
vend_b_missed_ded = vend_b_q['CALLS MISSED'].sum()

actual_total_missed = shared_missed_actual + vend_a_missed_ded + vend_b_missed_ded
miss_rate = actual_total_missed / total_calls * 100

total_calls, total_handled, actual_total_missed, miss_rate


(4804644.0, 4645473.0, 159171.0, 3.3128573105520407)

In [7]:
# Vendor-level volumes
vendor_summary = data.groupby('VENDOR_COMPANY').agg(
    calls_handled=('CALLS HANDLED', 'sum'),
    calls_missed=('CALLS MISSED', 'sum'),
    satisfied=('SATISFIED', 'sum'),
    calls_within_sla=('CALLS WITHIN SLA ', 'sum'),
    total_calls=('TOTAL_CALLS', 'sum')
)

# Rates
vendor_summary['satisfaction_rate'] = vendor_summary['satisfied'] / vendor_summary['calls_handled'] * 100
vendor_summary['sla_rate'] = vendor_summary['calls_within_sla'] / vendor_summary['total_calls'] * 100

vendor_summary


,calls_handled,calls_missed,satisfied,calls_within_sla,total_calls,satisfaction_rate,sla_rate
VENDOR_COMPANY,,,,,,,
Vendor A,2725531.5,101697.0,1017772,2.558861e+06,2827228.5,37.342148,90.507739
Vendor B,1919941.5,57474.0,758379,1.669114e+06,1977415.5,39.500110,84.408866


In [8]:
monthly_summary = (
    data.groupby(['MONTH', 'MONTH_NAME'])
        .agg(
            total_calls=('TOTAL_CALLS', 'sum'),
            calls_handled=('CALLS HANDLED', 'sum'),
            calls_missed=('CALLS MISSED', 'sum')
        )
        .reset_index()
        .sort_values('MONTH')
)

monthly_summary['miss_rate'] = monthly_summary['calls_missed'] / monthly_summary['total_calls'] * 100
monthly_summary


,MONTH,MONTH_NAME,total_calls,calls_handled,calls_missed,miss_rate
0,1,January,525312.0,509728.5,15583.5,2.966523
1,2,February,509694.0,493575.0,16119.0,3.162486
2,3,March,424036.5,411253.5,12783.0,3.014599
3,4,April,284634.0,275259.0,9375.0,3.293703
4,5,May,325135.5,314046.0,11089.5,3.410732
5,6,June,309541.5,299806.5,9735.0,3.144974
6,7,July,385971.0,372991.5,12979.5,3.362817
7,8,August,406243.5,391960.5,14283.0,3.515872
8,9,September,386244.0,374992.5,11251.5,2.913055
9,10,October,386946.0,375457.5,11488.5,2.969019


In [9]:
dow_summary = (
    data.groupby('DAY_OF_WEEK')
        .agg(
            total_calls=('TOTAL_CALLS', 'sum'),
            calls_handled=('CALLS HANDLED', 'sum'),
            calls_missed=('CALLS MISSED', 'sum'),
            satisfied=('SATISFIED', 'sum'),
            calls_within_sla=('CALLS WITHIN SLA ', 'sum')
        )
        .reset_index()
)

dow_summary['satisfaction_rate'] = dow_summary['satisfied'] / dow_summary['calls_handled'] * 100
dow_summary['sla_rate'] = dow_summary['calls_within_sla'] / dow_summary['total_calls'] * 100
dow_summary['miss_rate'] = dow_summary['calls_missed'] / dow_summary['total_calls'] * 100

dow_summary


,DAY_OF_WEEK,total_calls,calls_handled,calls_missed,satisfied,calls_within_sla,satisfaction_rate,sla_rate,miss_rate
0,Friday,721254.0,698647.5,22606.5,255347,633838.523843,36.548760,87.880071,3.134333
1,Monday,684493.5,662386.5,22107.0,272608,603161.380039,41.155428,88.117912,3.229687
2,Saturday,728715.0,702568.5,26146.5,279605,642342.228578,39.797543,88.147249,3.588028
3,Sunday,730203.0,705399.0,24804.0,249312,642788.596102,35.343401,88.028753,3.396864
4,Thursday,677407.5,653925.0,23482.5,257214,597093.276249,39.333869,88.143883,3.466525
5,Tuesday,628299.0,606333.0,21966.0,223558,552041.202195,36.870499,87.862817,3.496106
6,Wednesday,634272.0,616213.5,18058.5,238507,556709.382678,38.705254,87.771395,2.847122


In [10]:
# ==== Vendor-level summary ====

vendor_summary = (
    data.groupby('VENDOR_COMPANY')
        .agg(
            calls_handled=('CALLS HANDLED', 'sum'),
            calls_missed_raw=('CALLS MISSED', 'sum'),
            satisfied=('SATISFIED', 'sum'),
            calls_within_sla=('CALLS WITHIN SLA ', 'sum'),
            total_calls=('TOTAL_CALLS', 'sum')
        )
)

# Satisfaction & SLA rates
vendor_summary['satisfaction_rate'] = vendor_summary['satisfied'] / vendor_summary['calls_handled'] * 100
vendor_summary['sla_rate'] = vendor_summary['calls_within_sla'] / vendor_summary['total_calls'] * 100

vendor_summary


,calls_handled,calls_missed_raw,satisfied,calls_within_sla,total_calls,satisfaction_rate,sla_rate
VENDOR_COMPANY,,,,,,,
Vendor A,2725531.5,101697.0,1017772,2.558861e+06,2827228.5,37.342148,90.507739
Vendor B,1919941.5,57474.0,758379,1.669114e+06,1977415.5,39.500110,84.408866


In [11]:
# ==== Monthly summary for 2024 ====

monthly_summary = (
    data.groupby(['MONTH', 'MONTH_NAME'])
        .agg(
            total_calls=('TOTAL_CALLS', 'sum'),
            calls_handled=('CALLS HANDLED', 'sum'),
            calls_missed_raw=('CALLS MISSED', 'sum')
        )
        .reset_index()
        .sort_values('MONTH')
)

monthly_summary['miss_rate'] = monthly_summary['calls_missed_raw'] / monthly_summary['total_calls'] * 100

monthly_summary


,MONTH,MONTH_NAME,total_calls,calls_handled,calls_missed_raw,miss_rate
0,1,January,525312.0,509728.5,15583.5,2.966523
1,2,February,509694.0,493575.0,16119.0,3.162486
2,3,March,424036.5,411253.5,12783.0,3.014599
3,4,April,284634.0,275259.0,9375.0,3.293703
4,5,May,325135.5,314046.0,11089.5,3.410732
5,6,June,309541.5,299806.5,9735.0,3.144974
6,7,July,385971.0,372991.5,12979.5,3.362817
7,8,August,406243.5,391960.5,14283.0,3.515872
8,9,September,386244.0,374992.5,11251.5,2.913055
9,10,October,386946.0,375457.5,11488.5,2.969019


In [12]:
# ==== Day-of-week summary ====

dow_summary = (
    data.groupby('DAY_OF_WEEK')
        .agg(
            total_calls=('TOTAL_CALLS', 'sum'),
            calls_handled=('CALLS HANDLED', 'sum'),
            calls_missed_raw=('CALLS MISSED', 'sum'),
            satisfied=('SATISFIED', 'sum'),
            calls_within_sla=('CALLS WITHIN SLA ', 'sum')
        )
        .reset_index()
)

dow_summary['satisfaction_rate'] = dow_summary['satisfied'] / dow_summary['calls_handled'] * 100
dow_summary['sla_rate'] = dow_summary['calls_within_sla'] / dow_summary['total_calls'] * 100
dow_summary['miss_rate'] = dow_summary['calls_missed_raw'] / dow_summary['total_calls'] * 100

dow_summary


,DAY_OF_WEEK,total_calls,calls_handled,calls_missed_raw,satisfied,calls_within_sla,satisfaction_rate,sla_rate,miss_rate
0,Friday,721254.0,698647.5,22606.5,255347,633838.523843,36.548760,87.880071,3.134333
1,Monday,684493.5,662386.5,22107.0,272608,603161.380039,41.155428,88.117912,3.229687
2,Saturday,728715.0,702568.5,26146.5,279605,642342.228578,39.797543,88.147249,3.588028
3,Sunday,730203.0,705399.0,24804.0,249312,642788.596102,35.343401,88.028753,3.396864
4,Thursday,677407.5,653925.0,23482.5,257214,597093.276249,39.333869,88.143883,3.466525
5,Tuesday,628299.0,606333.0,21966.0,223558,552041.202195,36.870499,87.862817,3.496106
6,Wednesday,634272.0,616213.5,18058.5,238507,556709.382678,38.705254,87.771395,2.847122


In [13]:
# ==== Export key summary tables for reuse (e.g., Tableau, other notebooks) ====

output_dir = "data"

vendor_summary.to_csv(f"{output_dir}/vendor_summary_2024.csv", index=True)
monthly_summary.to_csv(f"{output_dir}/monthly_summary_2024.csv", index=False)
dow_summary.to_csv(f"{output_dir}/dow_summary_2024.csv", index=False)

print("Exported: vendor_summary_2024.csv, monthly_summary_2024.csv, dow_summary_2024.csv")


Exported: vendor_summary_2024.csv, monthly_summary_2024.csv, dow_summary_2024.csv
